##**(데이터 불러오기 방법 - 1) 로컬 컴퓨터 위치에 저장되어 있는 파일 Colab으로 저장하기**

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
dataset = pd.read_csv("파일명")
print(dataset)
print("데이터 크기(레코드, 필드): {}".format(dataset.shape))

##**(데이터 불러오기 방법 - 2) 파일을 구글 드라이브에 업로드 후, Colab 연결하여 불러오기**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import pandas as pd
dataset = pd.read_csv("/content/gdrive/My Drive/파일명")
print(dataset)
print("데이터 크기(레코드, 필드): {}".format(dataset.shape))

#**Ch5_의사결정 나무 실습**

##**의사결정나무 생성을 위해 필요한 라이브러리 불러오기**

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import matplotlib.pyplot as plt

##**데이터 불러오기(scikit-learn 라이브러리 제공 데이터)**

In [ ]:
cancer = load_breast_cancer()
print(cancer)

##**데이터 확인**

In [ ]:
import pandas as pd
df = pd.DataFrame(data=cancer.data, columns=cancer.feature_names)
df['class']=cancer.target
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
print("데이터 크기(레코드, 필드): {}".format(cancer.data.shape))
print("필드명 목록:\n{}".format(cancer.feature_names))

##**훈련용(80%), 테스트용(20%)으로 분류, 각 데이터 집합별 레코드 수 확인**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, 
                                                    stratify=cancer.target, test_size=0.2, random_state=1000)
print("훈련용 데이터 크기(레코드, 필드): {}".format(X_train.shape))
print("테스트용 데이터 크기(레코드, 필드): {}".format(X_test.shape))

##**의사결정나무 생성하기 1 - 기본**

###**의사결정나무 생성**

In [ ]:
tree_1 = DecisionTreeClassifier(random_state=0)
tree_1.fit(X_train, y_train)

In [ ]:
tree_1.feature_importances_

###**의사결정나무 결과 시각화**

**의사결정나무 시각화를 위한 라이브러리 불러오기**

In [ ]:
from sklearn.tree import export_graphviz
import graphviz

**의사결정나무 시각화 서식 생성**

In [ ]:
export_graphviz(tree_1, out_file="tree_1.dot", class_names=["M(Malignant Tumor)", "B(Benign Tumor)"], feature_names=cancer.feature_names, impurity=False, filled=True)

**의사결정나무 시각화 실행**

In [ ]:
with open("tree_1.dot", encoding='UTF8') as f:
    dot_graph = f.read()
    decesion_tree = graphviz.Source(dot_graph)
    display(decesion_tree)

**의사결정나무 시각화 저장**

In [ ]:
graphviz.Source(dot_graph, format='png').view()

###**의사결정나무 변수별 중요도 확인**

**의사결정나무의 중요 변수를 확인하기 위한 함수 정의**

In [ ]:
def plot_feature_importances_cancer(model):
     n_features = cancer.data.shape[1]
     plt.figure(figsize=(10, 10))
     plt.barh(range(n_features), model.feature_importances_, align='center')
     plt.yticks(np.arange(n_features), cancer.feature_names)
     plt.xlabel("Importance by Features")
     plt.ylabel("Features")
     plt.ylim(-1, n_features)
     plt.show()

**변수별 중요도 시각화 하기**

In [ ]:
plot_feature_importances_cancer(tree_1)

###**의사결정나무 모형의 성능평가**

**훈련, 테스트 집합 정확도 확인**

In [ ]:
print("훈련 데이터 집합 정확도: {:.3f}".format(tree_1.score(X_train, y_train)))
print("테스트 데이터 집합 정확도: {:.3f}".format(tree_1.score(X_test, y_test)))

**생성된 의사결정나무를 활용하여 테스트 데이터에 대한 분류**

In [ ]:
y_pred_1 = tree_1.predict(X_test)
print(y_pred_1)

**의사결정나무의 분류 결과와 테스트 데이터의 실제 값을 활용하여 교차표 생성**

In [ ]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report

results_1 = confusion_matrix(y_test, y_pred_1)
print(results_1)

**의사결정나무의 분류 결과와 테스트 데이터의 실제 값을 활용하여 모델의 성능평가 지표 출력**

In [ ]:
print('Confusion Matrix :\n{}'.format(results_1))

In [ ]:
print('Accuracy Score : {}'.format(accuracy_score(y_test, y_pred_1)))
print(classification_report(y_test, y_pred_1, target_names=['class 0', 'class 1']))

##**의사결정나무 생성하기 2 - 나무의 깊이 4**

###**의사결정나무 생성 -나무의 최대 깊이 4**

In [ ]:
tree_2 = DecisionTreeClassifier(max_depth=4, random_state=0)
tree_2.fit(X_train, y_train)

https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

###**의사결정나무 결과 시각화 - 최대 나무 깊이 4**

In [ ]:
export_graphviz(tree_2, out_file="tree_2.dot", class_names=["M(Malignant Tumor)", "B(Benign Tumor)"], feature_names=cancer.feature_names, impurity=False, filled=True)

In [ ]:
with open("tree_2.dot", encoding='UTF8') as f:
    dot_graph = f.read()
    decesion_tree = graphviz.Source(dot_graph)
    display(decesion_tree)

###**의사결정나무 변수별 중요도 확인 - 최대 나무 깊이 4**

In [ ]:
plot_feature_importances_cancer(tree_2)

###**의사결정나무 모형의 성능 평가  - 최대 나무 깊이 4**

**의사결정나무(최대 나무 깊이 4)의 훈련, 테스트 데이터 분류 정확도 확인하기**

In [ ]:
print("훈련 세트 정확도: {:.3f}".format(tree_2.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(tree_2.score(X_test, y_test)))

**생성된 의사결정나무(최대 나무 깊이 4)를 활용하여 암 여부 분류하기**

In [ ]:
y_pred_2 = tree_2.predict(X_test)
y_pred_2

**의사결정나무(최대 나무 깊이 4)의 분류 결과와 테스트 데이터의 실제 값을 활용하여 교차표 생성**

In [ ]:
results_2 = confusion_matrix(y_test, y_pred_2)
print(results_2)

**의사결정나무(최대 나무 깊이 4)의 분류 결과와 테스트 데이터의 실제 값을 활용하여 모델의 성능평가 지표 출력**

In [ ]:
print('Confusion Matrix :\n{}'.format(results_2))
print('Accuracy Score : {}'.format(accuracy_score(y_test, y_pred_2)))
print(classification_report(y_test, y_pred_2, target_names=['class 0', 'class 1']))

##**의사결정나무 가지치기**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, 
                                                    stratify=cancer.target, test_size=0.2, random_state=1000)

In [ ]:
score_train = []
score_test = []

for depth in range(1,15):
    model = DecisionTreeClassifier(max_depth=depth)
    model.fit(X_train, y_train)
    score1 = model.score(X_train,y_train)
    score2 = model.score(X_test, y_test)
    score_train.append(score1)
    score_test.append(score2)

In [ ]:
score_test

In [ ]:
plt.plot(range(1,15), score_train, 'ro--')
plt.plot(range(1,15), score_test, 'bs-')
plt.legend(['train','test'])
plt.xlabel('max_depth')